In [1]:
from osgeo import gdal, osr
import os
from pathlib import Path
import numpy as np
import rasterio as rio
from rasterio.plot import show
import re
from multiprocessing import Pool

hansen_path_in = Path('/home/rstudio/data/hansen_raw')
hansen_path_out = Path('/home/rstudio/data/hansen_250m')

sample_rast_path = Path('/home/rstudio/data/impacts_data/covariates_lc_2015.tif')

In [7]:
%%writefile parallel_functions.py

import numpy as np
from osgeo import gdal, osr
from pathlib import Path
import re
import os


def get_tile_coords(f):
    '''returns tile coords part of Hansen GFC file name'''
    return re.search('[0-9]*[NS]_[0-9]*[EW]', f)[0]


def get_lossname(cover_file):
    '''returns name of the loss year file associated with a particular cover file'''
    tile_coords = re.search('[0-9]*[NS]_[0-9]*[EW]', cover_file)[0]
    return('Hansen_GFC-2020-v1.8_lossyear_' + get_tile_coords(cover_file) + '.tif')


def recode_cover_file(
    arguments: dict
):
    cover_file = arguments['cover_file']
    out_folder = arguments['out_folder']
    xres = arguments['xres']
    yres = arguments['yres']
    
    out_file = str(
        out_folder / Path(
            'Hansen_GFC-2020-v1.8_250m_coverbyyear_' + get_tile_coords(str(cover_file.name)) + '.tif'
        )
    )
    
    if os.path.exists(out_file):
        return None
    
    cover_ds = gdal.Open(str(cover_file))
    cover_band = cover_ds.GetRasterBand(1)

    cover = cover_band.ReadAsArray()
    cover = cover > 30

    loss_ds = gdal.Open(str(cover_file.parent / get_lossname(str(cover_file.name))))
    loss_band = loss_ds.GetRasterBand(1)
    loss = loss_band.ReadAsArray()

    driver = gdal.GetDriverByName("MEM")
    mem_ds = driver.Create(
        '',
        cover_band.XSize,
        cover_band.YSize,
        21,
        gdal.GDT_Byte
    )
    src_gt = cover_ds.GetGeoTransform()
    mem_ds.SetGeoTransform(src_gt)
    dst_srs = osr.SpatialReference()
    dst_srs.ImportFromWkt(cover_ds.GetProjectionRef())
    mem_ds.SetProjection(dst_srs.ExportToWkt())
    # Threshold forest to be greater than 30% cover
    for n in range(21):
        mem_ds.GetRasterBand(n + 1).WriteArray(cover * np.logical_or(loss == 0, loss > n) * 100)


    gdal.Warp(
        out_file,
        mem_ds,
        xRes=xres,
        yRes=yres,
        resampleAlg='average',
        options=['COMPRESS=LZW'],
        multithread=True,
        dstSRS="epsg:4326",
        outputType=gdal.GDT_Byte
    )

Overwriting parallel_functions.py


In [8]:
%%time

import parallel_functions

sample_rast_path = Path('/home/rstudio/data/impacts_data/covariates_lc_2015.tif')
ds = gdal.Open(str(sample_rast_path))
gt = ds.GetGeoTransform()
xres = gt[1]
yres = gt[5]

cover_files = [p for p in hansen_path_in.glob('*treecover2000*.tif')]

arguments_list = []
for cover_file in cover_files:
    arguments = {
        'cover_file': cover_file,
        'out_folder': Path('/home/rstudio/data/hansen_250m'),
        'xres': xres,
        'yres': yres
    }
    arguments_list.append(arguments)

with Pool(30) as p:
    p.map(parallel_functions.recode_cover_file, arguments_list, chunksize=10)

#parallel_functions.recode_cover_file(
#    hansen_path_in / "Hansen_GFC-2020-v1.8_treecover2000_00N_020E.tif",
#    Path('/home/rstudio/data/hansen_250m'),
#    xres,
#    yres
#)


CPU times: user 0 ns, sys: 164 ms, total: 164 ms
Wall time: 197 ms


In [ ]:
import parallel_functions
sample_rast_path = Path('/home/rstudio/data/impacts_data/covariates_lc_2015.tif')
ds = gdal.Open(str(sample_rast_path))
gt = ds.GetGeoTransform()
xres = gt[1]
yres = gt[5]

a = parallel_functions.recode_cover_file(
    {
        'cover_file': hansen_path_in / "Hansen_GFC-2020-v1.8_treecover2000_00N_020E.tif",
        'out_folder':  Path('/home/rstudio/data/hansen_250m'),
        'xres': xres,
        'yres': yres
    }
)
print (a)

In [ ]:
cover_vrt = str(hansen_path_out / 'Hansen_GFC-2020-v1.8_treecover2000.vrt')
gdal.BuildVRT(
    cover_vrt,
    [str(p) for p in hansen_path_in.glob('*treecover2000*.tif')]
)
loss_vrt = str(hansen_path_out / 'Hansen_GFC-2020-v1.8_lossyear.vrt')
out = gdal.BuildVRT(
    loss_vrt,
    [str(p) for p in hansen_path_in.glob('*lossyear*tif')]  
)

In [5]:
arguments_list

[{'cover_file': PosixPath('/home/rstudio/data/hansen_raw/Hansen_GFC-2020-v1.8_treecover2000_00N_030W.tif'),
  'out_folder': PosixPath('/home/rstudio/data/hansen_250m'),
  'xres': 0.008333329999999998,
  'yres': -0.00833333},
 {'cover_file': PosixPath('/home/rstudio/data/hansen_raw/Hansen_GFC-2020-v1.8_treecover2000_00N_050E.tif'),
  'out_folder': PosixPath('/home/rstudio/data/hansen_250m'),
  'xres': 0.008333329999999998,
  'yres': -0.00833333},
 {'cover_file': PosixPath('/home/rstudio/data/hansen_raw/Hansen_GFC-2020-v1.8_treecover2000_00N_080E.tif'),
  'out_folder': PosixPath('/home/rstudio/data/hansen_250m'),
  'xres': 0.008333329999999998,
  'yres': -0.00833333},
 {'cover_file': PosixPath('/home/rstudio/data/hansen_raw/Hansen_GFC-2020-v1.8_treecover2000_00N_040E.tif'),
  'out_folder': PosixPath('/home/rstudio/data/hansen_250m'),
  'xres': 0.008333329999999998,
  'yres': -0.00833333},
 {'cover_file': PosixPath('/home/rstudio/data/hansen_raw/Hansen_GFC-2020-v1.8_treecover2000_00N_010W